# Environment Setup

In [2]:
# Enable autoreload of libraries on execution
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

In [3]:
from IPython import get_ipython

def is_using_colab() -> bool:
    return "google.colab" in str(get_ipython())

In [4]:
if is_using_colab():
  import os
  import zipfile
  from google.colab import drive
  os.system("pip -q install torchinfo")
  os.system("pip -q install pytorch-lightning")
  os.system("pip -q install torchmetrics[image]")

  !mkdir Experiments
  !mkdir Datasets
  
  # Copy code and datasets zips
  drive.mount('/content/drive')
  !cp -r "/content/drive/MyDrive/Master Thesis/02 - Code/." "."
  # !cp -r "/content/drive/MyDrive/Master Thesis/01 - Datasets/." "Datasets/."
  !cp -r "/content/drive/MyDrive/Master Thesis/01 - Datasets/DIV2K_800_5.zip" "Datasets/DIV2K_800_5.zip"

  # Extract datasets zips
  for zip in os.listdir("Datasets"):
    with zipfile.ZipFile(os.path.join("Datasets", zip), 'r') as zip_ref:
      zip_ref.extractall(os.path.join("Datasets", zip.replace(".zip", "")))
    os.remove(os.path.join("Datasets", zip))

  device = "cuda"
  experiments_dir = "Experiments"
  datasets_path = r"Datasets/"
else:
  device = "cuda"
  experiments_dir = "../05 - Experiments"
  datasets_path = r"C:\datasets"

In [5]:
import os
import time
import gc
import json

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display

from media_utils import plot_img

# Dataset Exploration


In [5]:
# dataset_path = r"G:\VM\Shared Folder\bags\0001.bag"
# EVENTS_TOPIC = "/cam0/events"
# IMAGES_TOPIC = "/cam0/image_raw"

dataset_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\03 - Dataset\CED_simple\simple_jenga_destroy.bag"
EVENTS_TOPIC = "/dvs/events"
IMAGES_TOPIC = "/dvs/image_color"

## Dataset Inspection


### Inspect messages content


In [ ]:
from dataset_utils import inspect_bag
inspect_bag(dataset_path)

##### **Image events content:**

**data**: binary image data encoded according to encoding

**encoding**: format used to encode the image data - e.g. mono8, bgr8

**width**, **height**: size of the image

**is_bigendian**: endianness of image

**step**: full row length in bytes

**header**:

-   **seq**: consecutively increasing ID
-   **stamp**:
    -   **secs**: seconds since epoch
    -   **nsecs**: nanoseconds since stamp.secs
-   **frame_id**: frame this data is associated with - in this dataset is always empty

##### **Event events content:**

**events**: array of events

**width**, **height**: size of the camera sensor

**header**:

-   **seq**: consecutively increasing ID
-   **stamp**:
    -   **secs**: seconds since epoch
    -   **nsecs**: nanoseconds since stamp.secs
-   **frame_id**: frame this data is associated with - in this dataset is always empty

##### **Events format:**

**x**, **y**: coordinates of the pixel sensor

**polarity**: polarity of the event in {True, False}

**ts**:

-   **secs**: seconds since epoch
-   **nsecs**: nanoseconds since ts.secs


### Inspect message timestamps


In [ ]:
from dataset_utils import inspect_message_timestamps
inspect_message_timestamps(dataset_path, events_topic=EVENTS_TOPIC, images_topic=IMAGES_TOPIC)


The events are not completely sequential, since there can be two image events after each other (e.g. 6-7) but the first event timestamp of the next event is before the last image (e.g 8).

Moreover, there are some events that are acquired after the last image has been captured.


## Dataset Analysis

### Dataset Loading

In [ ]:
from dataset_utils import load_bag_as_dataframes
events_df, images_df = load_bag_as_dataframes(dataset_path, events_topic=EVENTS_TOPIC, images_topic=IMAGES_TOPIC, max_events=1e6)
print("Events dataframe")
display(events_df)
print("Images dataframe")
display(images_df)


Events and frames are already ordered by timestamp, since they are produced from a real simulation.


### Number of events per image frame

Check how many events there are among each image frame.

This roughly reflects the amount of movement in the sequence.


In [ ]:
from dataset_utils import plot_number_of_events_per_frame
plot_number_of_events_per_frame(events_df, images_df)

### Check image frames frequency


In [ ]:
from dataset_utils import plot_image_frames_frequency
plot_image_frames_frequency(images_df)

### Save events for visualization


In [ ]:
from media_utils import save_visual_bayer_events, save_visual_accumulated_events

w, h = images_df.iloc[0].width, images_df.iloc[0].height
folder_name = os.path.basename(dataset_path).replace(".bag", "")
output_dir = os.path.join(os.path.dirname(dataset_path), folder_name)
save_visual_bayer_events(events_df, w, h, os.path.join(output_dir, "visual_events"))
save_visual_accumulated_events(events_df, w, h, os.path.join(output_dir, "visual_images"))

## Dataset Generation


In [ ]:
from dataset_utils import dataset_generator_from_batches
batches_path = os.path.join(dataset_path.replace(".bag", ""), "batches")
for batch in dataset_generator_from_batches(batches_path):

    # Old batch format
    (in_, events), out_ = batch
    plot_img(in_)
    print(events.shape)
    plot_img(out_)

    # New batch format
    # events, img = batch
    # print(events.shape)
    # plot_img(img)
    
    break


# Datasets Definition


In [6]:
DATASET_PARAMS = {
    "limit": None,
    "preload_to_RAM": True,
    "crop_size": (128, 128)
}

DATALOADER_PARAMS = {
    "batch_size": 8,
    "num_workers": 0,
    "pin_memory": True,
}

if is_using_colab():
    DATASET_PARAMS["preload_to_RAM"] = True
    DATALOADER_PARAMS["batch_size"] = 64
    DATALOADER_PARAMS["num_workers"] = 0

## CED Dataset

In [ ]:
CED_DATASET_PATH = os.path.join(datasets_path, "CEDDataset")
available_sequences = os.listdir(CED_DATASET_PATH)
print("Available sequences:")
for i, seq in enumerate(available_sequences):
    print(f"{i:<5}{seq}")

In [ ]:
from dataset import CEDDataset
train_datasets_names = ["simple_color_keyboard_1", "simple_fruit"]
train_dataset = CEDDataset(CED_DATASET_PATH, sequences=train_datasets_names, ignore_input_image=True, **DATASET_PARAMS)

valid_datasets_names = ["simple_rabbits"]
valid_dataset = CEDDataset(CED_DATASET_PATH, sequences=valid_datasets_names, ignore_input_image=True, **DATASET_PARAMS)

test_datasets_names = ["simple_color_keyboard_2", "simple_jenga_1", "simple_wires_1"]
test_dataset = CEDDataset(CED_DATASET_PATH, sequences=test_datasets_names, ignore_input_image=True, **DATASET_PARAMS)


## Synthetic Dataset from DIV2K

In [7]:
from dataset import DIV2KDataset
DIV2K_DATASET_PATH = os.path.join(datasets_path, "DIV2K_800_5")
train_datasets_names = ["{:04}".format(i) for i in range(1, 800 + 1)]
# train_datasets_names = ["{:04}".format(i) for i in range(1, 25 + 1)]
train_dataset = DIV2KDataset(DIV2K_DATASET_PATH, sequences=train_datasets_names, **DATASET_PARAMS)

valid_datasets_names = ["{:04}".format(i) for i in range(801, 900 + 1)]
# valid_datasets_names = ["{:04}".format(i) for i in range(801, 805 + 1)]
valid_dataset = DIV2KDataset(DIV2K_DATASET_PATH, sequences=valid_datasets_names, **DATASET_PARAMS)

In [8]:
def generate_batch_from_files(paths, pre_process_fn):
    X = []
    y = []
    for path in paths:
        sample = pre_process_fn(torch.load(path))
        X.append(torch.tensor(sample[0], dtype=torch.float32))
        y.append(torch.tensor(sample[1], dtype=torch.float32))
    batch = torch.stack(X), torch.stack(y)
    return batch


train_batch_paths = [
    os.path.join(DIV2K_DATASET_PATH, "0001", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0002", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0003", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0004", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0005", "batch_0000.pt"),
]
valid_batch_paths = [
    os.path.join(DIV2K_DATASET_PATH, "0801", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0802", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0803", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0804", "batch_0000.pt"),
    os.path.join(DIV2K_DATASET_PATH, "0805", "batch_0000.pt"),
]

train_batch = generate_batch_from_files(train_batch_paths, train_dataset.pre_process)
valid_batch = generate_batch_from_files(valid_batch_paths, valid_dataset.pre_process)

# PyTorch Dataset and DataLoader

In [9]:
# Check if there is no instersection between train and valid dataset
union = set(train_datasets_names).union(set(valid_datasets_names))
sum_of_lengths = sum([len(ds) for ds in [train_datasets_names, valid_datasets_names]])
assert len(union) == sum_of_lengths, "Some datasets are in common"

In [ ]:
from dataset import CEDDataset, ConcatBatchSampler

# Do not batch events coming from different datasets
# samplers = [torch.utils.data.RandomSampler(ds) for ds in train_datasets]
# sampler = ConcatBatchSampler(samplers=samplers, batch_size=TRAIN_DS_PARAMS["batch_size"], drop_last=False)
# train_dataloader = torch.utils.data.DataLoader(
#     concat_ds, batch_sampler=sampler,
# )

train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **DATALOADER_PARAMS)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, shuffle=False, **DATALOADER_PARAMS)

DATASET_PARAMS.update({
    "train_datasets_names": train_datasets_names, 
    "valid_datasets_names": valid_datasets_names, 
})

print("Train samples: {} \t Train batches: {:<10}".format(len(train_dataset), len(train_dataloader)))
print("Valid samples: {} \t Valid batches: {:<10}".format(len(valid_dataset), len(valid_dataloader)))

# Inspect datasets

In [ ]:
# Inspect train dataloader
from media_utils import save_video_tensors
frames = []

# Old dataset format
# for (i, e), o in tqdm(train_dataloader):

for e, o in tqdm(train_dataloader):
  for batch in o:
    frames.append(batch)

save_video_tensors("train_frames.mp4", frames, 5)

In [ ]:
# Inspect valid dataloader
from media_utils import save_video_tensors
frames = []

# Old dataset format
# for (i, e), o in tqdm(valid_dataloader):

for e, o in tqdm(valid_dataloader):
  for batch in o:
    frames.append(batch)

save_video_tensors("valid_frames.mp4", frames, 5)

In [ ]:
# Inspect test dataloader
from media_utils import save_video_tensors
frames = []

# Old dataset format
# for (i, e), o in tqdm(test_dataloader):

for e, o in tqdm(test_dataloader):
  for batch in o:
    frames.append(batch)

save_video_tensors("test_frames.mp4", frames, 5)

# UNet AutoEncoder Model


In [ ]:
from torchinfo import summary
from models import EventsToImagesUNet

PARAMS = {
    "input_channels": 3 + 10,
}

model = EventsToImagesUNet(PARAMS["input_channels"])
# summary(model, input_size=(4, input_channels, 256, 336), device=device)


## Model Training


In [ ]:
from train import train_unet

PARAMS.update({
    "n_epochs": 3,
    "learning_rate": 0.0001,
    "comment": "",
})
PARAMS.update({"train_dataset_params": DATASET_PARAMS})

train_unet(model, device, train_dataloader, PARAMS, log_path=experiments_dir, valid_ds=valid_dataloader, save_best_model=True)

In [ ]:
torch.save(model.state_dict(), "model_thesis.pt")

In [ ]:
# Save input frames as video
from media_utils import save_video_tensors
imgs = []
for batch in tqdm(train_dataloader):
    (input_images, events_tensors), ground_truth_images = batch
    imgs += input_images
save_video_tensors("input_video.mp4", imgs, 30)


In [ ]:
# Save generated frames as video
use_prev_images_as_input = False

imgs = []
for i, batch in enumerate(tqdm(train_dataloader)):
    if use_prev_images_as_input and i != 0:
        input_images = generated_images
    else:
        (input_images, events_tensors), ground_truth_images = batch
    input_images = torch.einsum("bhwc -> bchw", input_images)

    input_tensors = torch.hstack((input_images, events_tensors))
    input_tensors = input_tensors.to(device)

    generated_images = model(input_tensors).cpu().detach()
    generated_images = torch.einsum("bchw -> bhwc", generated_images)

    imgs += generated_images

save_video_tensors("generated_video.mp4", imgs, 30)


In [ ]:
# BS = 1; Seconds for one epoch: 142.36227083206177
# BS = 2; Seconds for one epoch: 132.93124723434448
# BS = 4; Seconds for one epoch: 127.95218682289124
# BS = 8; Seconds for one epoch: 138.49955368041992
# BS = 16; Seconds for one epoch: after 180 seconds it was at 3/10 batches so I stopped it


# Transformer Model


## Model definition


In [ ]:
from models import TransformerModel

PARAMS = {
    "input_shape": (336, 256, 3),
    "encoding_size": 512,
    "heads": 4,
    "layers_number": 2,
}

model = TransformerModel(PARAMS["input_shape"], PARAMS["encoding_size"], PARAMS["heads"], PARAMS["layers_number"])

## Model Training


In [ ]:
from train import train_transformer

PARAMS.update({
    "n_epochs": 3,
    "learning_rate": 0.0001,
    "comment": "",
})
PARAMS.update({"train_dataset_params": DATASET_PARAMS})

train_transformer(model, device, train_dataloader, PARAMS, log_path=experiments_dir, valid_ds=valid_dataloader, save_best_model=True)

In [ ]:
# Save model
torch.save(model.state_dict(), "transformer.pt")

In [ ]:
# Load Model
model.load_state_dict(torch.load("transformer.pt"))

# Autoencoder Model

In [11]:
from models.autoencoder import EventEncoder, EventDecoder, EventAutoEncoder
from torchinfo import summary

PARAMS = {
    "n_filters": [16, 16, 32, 32],
    "input_features": 1,
    "lr": 0.001
}

ee = EventEncoder(PARAMS["input_features"], PARAMS["n_filters"])
ed = EventDecoder(PARAMS["n_filters"][-1], PARAMS["n_filters"][::-1])
model = EventAutoEncoder(ee, ed, PARAMS["lr"])

# summary(eae, input_size=(8, 1, 128, 128), device=device)

In [12]:
train_events = train_batch[0][:, 1]
train_batch = (train_events.unsqueeze(1), train_events)
valid_events = valid_batch[0][:, 1]
valid_batch = (valid_events.unsqueeze(1), valid_events)

In [ ]:
from utils import LogImagesCallback, KerasProgressBar, ColabSaveCallback
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

PARAMS.update({
    "n_epochs": 300,
    "comment": "Large - Event AE, changed training set"
})
PARAMS.update({"dataset_params": DATASET_PARAMS})
PARAMS.update({"dataloader_params": DATALOADER_PARAMS})


callbacks = []
callbacks.append(LogImagesCallback(train_batch, valid_batch, n=5, n_epochs=5))
callbacks.append(KerasProgressBar())
if is_using_colab():
    dst_path = "/content/drive/MyDrive/Master Thesis"
    colab_cb = ColabSaveCallback("Exp.zip", dst_path, 60*60, ["zip -r Exp.zip Experiments"])
    callbacks.append(colab_cb)

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_last=True)
checkpoint_callback.CHECKPOINT_NAME_LAST = "{epoch}-last"
callbacks.append(checkpoint_callback)

logger = pl.loggers.TensorBoardLogger(experiments_dir, version=PARAMS["comment"])
log_every = 50
if len(train_dataloader) < log_every:
    log_every = 1

params_markdown = "```json\n" + json.dumps(PARAMS, indent=2).replace("\n","  \n") + "\n```"
logger.experiment.add_text("params", params_markdown)

trainer = pl.Trainer(max_epochs=PARAMS["n_epochs"], callbacks=callbacks, accelerator="gpu", profiler=None, logger=logger, log_every_n_steps=log_every)
trainer.fit(model, train_dataloader, valid_dataloader)

# Event Encoder Transformer Model

In [ ]:
from models import EventEncoderTransformer
from torchinfo import summary

PARAMS = {
    "output_shape": (336, 256, 3),
    "encoding_size": 336,
    "heads": 4,
    "layers_number": 6
}

# The model is going to use the previous AutoEncoder, so be sure to use a pre-trained version of it

model = EventEncoderTransformer(
    output_shape=PARAMS["output_shape"], 
    encoder=eae.encoder, 
    encoding_size=PARAMS["encoding_size"], 
    heads=PARAMS["heads"], 
    layers_number=PARAMS["layers_number"]
)

# summary(model, input_size=(4, 10, 256, 336))


In [ ]:
from train import train_transformer

PARAMS.update({
    "n_epochs": 300,
    "learning_rate": 0.0001,
    "comment": ""
})
PARAMS.update({"train_dataset_params": DATASET_PARAMS})

train_transformer(model, device, train_dataloader, PARAMS, log_path=experiments_dir, valid_ds=valid_dataloader, save_best_model=True)

In [ ]:
# Save model
torch.save(model.state_dict(), "model_tae.pt")

In [ ]:
# Load model
model.load_state_dict(torch.load("model_tae.pt"))

# ViT-like Model

In [ ]:
# Clean VRAM
del model
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
import pytorch_lightning as pl
from models.transformer import VisionTransformer

PARAMS = {
    "input_shape": (10, 128, 128),
    "patch_size": (32, 32),
    "encoding_size": 256,
    "heads": 4,
    "layers_number": 3,
    "use_linear_proj": True,
    "learning_rate": 0.0001,
    "use_LPIPS": False,
    "vgg_layer": "features.35",
}
model = VisionTransformer(**PARAMS)

In [ ]:
if is_using_colab():
    from IPython import get_ipython
    get_ipython().magic('tensorboard --logdir "Experiments/lightning_logs"')

In [ ]:
from utils import LogImagesCallback, KerasProgressBar
from pytorch_lightning.callbacks import ModelCheckpoint

PARAMS.update({
    "n_epochs": 2,
    "comment": "Test with VGG54"
})
PARAMS.update({"train_dataset_params": DATASET_PARAMS})

callbacks = []

train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))
callbacks.append(LogImagesCallback(train_batch, valid_batch, n=5, n_epochs=5))

callbacks.append(KerasProgressBar())

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_last=True)
checkpoint_callback.CHECKPOINT_NAME_LAST = "{epoch}-last"
callbacks.append(checkpoint_callback)

logger = pl.loggers.TensorBoardLogger(experiments_dir, version=PARAMS["comment"])
log_every = 50
if len(train_dataloader) < log_every:
    log_every = 1

params_markdown = "```json\n" + json.dumps(PARAMS, indent=2).replace("\n","  \n") + "\n```"
logger.experiment.add_text("params", params_markdown)

trainer = pl.Trainer(max_epochs=PARAMS["n_epochs"], callbacks=callbacks, accelerator="gpu", profiler=None, logger=logger, log_every_n_steps=log_every)
trainer.fit(model, train_dataloader, valid_dataloader)

In [ ]:
# Save model
torch.save(model.state_dict(), "model_vit.pt")

In [ ]:
# Load model
model.load_state_dict(torch.load("model_vit.pt"))

# Conv ViT

In [ ]:
# Clean VRAM
del model
torch.cuda.empty_cache()
import gc
gc.collect()

In [ ]:
import pytorch_lightning as pl
from models.transformer import VisionTransformerConv

PARAMS = {
    "input_shape": (10, 128, 128),
    "patch_size": (32, 32),
    "heads": 4,
    "layers_number": 1,
    "learning_rate": 0.0001,
    "image_loss_weight": 1,
    "feature_loss_weight": 1e-2,
}
model = VisionTransformerConv(**PARAMS)

In [ ]:
from torchinfo import summary
summary(model, input_size=(8, 10, 128, 128), col_names=["input_size", "output_size", "num_params"])

In [ ]:
from utils import LogImagesCallback, KerasProgressBar, ColabSaveCallback
from pytorch_lightning.callbacks import ModelCheckpoint

PARAMS.update({
    "n_epochs": 1000,
    "comment": "Large, Long - 1 il, 1e-2 fl, bn relu, maxpool"
})
PARAMS.update({"dataset_params": DATASET_PARAMS})
PARAMS.update({"dataloader_params": DATALOADER_PARAMS})


callbacks = []
callbacks.append(LogImagesCallback(train_batch, valid_batch, n=5, n_epochs=5))
callbacks.append(KerasProgressBar())
if is_using_colab():
    dst_path = "/content/drive/MyDrive/Master Thesis"
    colab_cb = ColabSaveCallback("Exp.zip", dst_path, 60*60, ["zip -r Exp.zip Experiments"])
    callbacks.append(colab_cb)

checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_last=True)
checkpoint_callback.CHECKPOINT_NAME_LAST = "{epoch}-last"
callbacks.append(checkpoint_callback)

logger = pl.loggers.TensorBoardLogger(experiments_dir, version=PARAMS["comment"])
log_every = 50
if len(train_dataloader) < log_every:
    log_every = 1

params_markdown = "```json\n" + json.dumps(PARAMS, indent=2).replace("\n","  \n") + "\n```"
logger.experiment.add_text("params", params_markdown)

trainer = pl.Trainer(max_epochs=PARAMS["n_epochs"], callbacks=callbacks, accelerator="gpu", profiler=None, logger=logger, log_every_n_steps=log_every)
trainer.fit(model, train_dataloader, valid_dataloader)

In [ ]:
!zip -r Exp.zip "Experiments"
!mv "Exp.zip"  "/content/drive/MyDrive/Master Thesis"

# Evaluation

In [ ]:
from models.transformer import VisionTransformerConv
checkpoint_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\05 - Experiments\lightning_logs\1e-2 FL Long training\checkpoints\epoch=465-step=115102.ckpt"
model = VisionTransformerConv.load_from_checkpoint(checkpoint_path, feature_loss_weight=None, map_location="cuda")

In [ ]:
from utils import LogImagesCallback

def fig_to_numpy(fig):
    canvas = fig.canvas 
    canvas.draw()
    rgba = np.asarray(canvas.buffer_rgba())
    return rgba

def create_figure(batch, model):
    cb = LogImagesCallback(train_batch, valid_batch)
    x, y = batch
    outputs = model(x)[0]
    figs = []
    for i in range(len(x)):
        fig = cb._create_plot(outputs[i], y[i])
        figs.append(fig_to_numpy(fig))
        plt.close(fig)
    return np.hstack(figs)

path_split = checkpoint_path.split(os.sep)
RUN_NAME = path_split[path_split.index("checkpoints") - 1]
plt.figure(figsize=(20, 5))
plt.axis("off")
train_fig = create_figure(train_batch, model)
plt.title(f"{RUN_NAME} | Train")
plt.imsave(f"{RUN_NAME}_train.png", train_fig)
plt.imshow(train_fig)

plt.figure(figsize=(20, 5))
plt.axis("off")
plt.title(f"{RUN_NAME} | Valid")
valid_fig = create_figure(valid_batch, model)
plt.imsave(f"{RUN_NAME}_valid.png", valid_fig)
plt.imshow(valid_fig)

In [ ]:
# Save video results on train and test set
from media_utils import save_predicted_video
save_predicted_video(model, device, train_dataloader, "train_prediction.mp4")
save_predicted_video(model, device, test_dataloader, "test_prediction.mp4")

In [ ]:
from media_utils import save_events_frames_visualization
sensor_size = (128, 128)
save_events_frames_visualization(sensor_size, "train_inspect.mp4", train_dataloader, model=model, fps=15)
save_events_frames_visualization(sensor_size, "test_inspect.mp4", test_dataloader, model=model, fps=15)


In [ ]:
from media_utils import plot_square, predict_n_images

N_EVAL_IMAGES = 16

results = predict_n_images(train_dataset, N_EVAL_IMAGES, model)
plot_square(results, size=3)

results = predict_n_images(test_dataset, N_EVAL_IMAGES, model)
plot_square(results, size=3)

# Metadata parsing

In [ ]:
import json

metadata_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\05 - Experiments\2022-08-31 15-36-51\metadata.json"
with open(metadata_path, encoding="utf8") as f:
    data = json.load(f)

In [ ]:
import matplotlib.pyplot as plt

print("Number of epochs:", len(data["epochs"]))
train_loss = [epoch["train_loss"] for epoch in data["epochs"]]
valid_loss = [epoch["valid_loss"] for epoch in data["epochs"]]
plt.plot(range(len(data["epochs"])), train_loss, color="blue", label="Training Loss")
plt.plot(range(len(data["epochs"])), valid_loss, color="orange", label="Valid loss")
plt.legend()
plt.show()

In [ ]:
from dataset_utils import dataset_generator_from_bag, save_events_frames_view
# path = r"G:\CED Datasets\simple_color_keyboard_2.bag"
# gen = dataset_generator_from_bag(path, "/dvs/events", "/dvs/image_color", min_n_events=10000, crop_size=(128, 128))
path = r"G:\VM\Shared Folder\bags\DIV2K_0.5\0804.bag"
gen = dataset_generator_from_bag(path, "/cam0/events", "/cam0/image_raw", crop_size=(128, 128))
save_events_frames_view("signore.mp4", gen, model=model, denorm=True)

# Tests

## RAM Image Callback Test

In [ ]:
import pytorch_lightning as pl
import torch

from utils import KerasProgressBar, LogImagesCallback

class UselessModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.param = torch.nn.parameter.Parameter(torch.tensor(0, dtype=torch.float32))

    def forward(self, x):
        return self.param

    def training_step(self, train_batch, batch_idx):
        return np.random.rand() - self.param

    def validation_step(self, val_batch, batch_idx):
        return np.random.rand() - self.param
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)


model = UselessModel()

callbacks = []
train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))
callbacks.append(LogImagesCallback(train_batch, valid_batch, n=5, n_epochs=1))
callbacks.append(KerasProgressBar())

logger = pl.loggers.TensorBoardLogger(experiments_dir, "test CB")
log_every = 50
if len(train_dataloader) < log_every:
    log_every = 1

trainer = pl.Trainer(max_epochs=500, callbacks=callbacks, accelerator="gpu", logger=logger, log_every_n_steps=log_every, enable_progress_bar=True)
trainer.fit(model, train_dataloader, valid_dataloader)

## LPIPS Test

In [ ]:
from torchvision.models import vgg19, VGG19_Weights
from scipy.ndimage import gaussian_filter
from torch.nn.functional import mse_loss
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
import torch

features_outputs = []

img_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\03 - Dataset\CED_simple\simple_color_keyboard_1\color_images\0.png"
img2_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\03 - Dataset\CED_simple\simple_color_keyboard_1\color_images\130.png"
img3_path = r"E:\Cartelle Personali\Fabrizio\Universita\Magistrale\Tesi\03 - Dataset\CED_simple\simple_fruit\color_images\0.png"
img = plt.imread(img_path)[:,:,:3]
img_noise = img * np.random.rand(*img.shape)
img_blur = gaussian_filter(img, sigma=(5, 5, 0))
img2 = plt.imread(img2_path)[:,:,:3]
img3 = plt.imread(img3_path)[:,:,:3]

plt.figure(figsize=(10, 5))
plt.imshow(np.hstack((img, img_noise, img_blur, img2, img3)))
plt.show()

img = torch.Tensor(img).permute(2, 0, 1).unsqueeze(0)
img_noise = torch.Tensor(img_noise).permute(2, 0, 1).unsqueeze(0)
img_blur = torch.Tensor(img_blur).permute(2, 0, 1).unsqueeze(0)
img2 = torch.Tensor(img2).permute(2, 0, 1).unsqueeze(0)
img3 = torch.Tensor(img3).permute(2, 0, 1).unsqueeze(0)

vgg_weights = VGG19_Weights.IMAGENET1K_V1
vgg = vgg19(weights=vgg_weights)
vgg.eval()
vgg_preprocess = vgg_weights.transforms()

# train_nodes, eval_nodes = get_graph_node_names(vgg)
# print(train_nodes)

VGG_LAYER = "features.35"
vgg_extractor = create_feature_extractor(vgg, [VGG_LAYER])

features_img = vgg_extractor(vgg_preprocess(img))[VGG_LAYER]
features_noise = vgg_extractor(vgg_preprocess(img_noise))[VGG_LAYER]
features_blur = vgg_extractor(vgg_preprocess(img_blur))[VGG_LAYER]
features_img2 = vgg_extractor(vgg_preprocess(img2))[VGG_LAYER]
features_img3 = vgg_extractor(vgg_preprocess(img3))[VGG_LAYER]

mse_noise = mse_loss(img, img_noise)
mse_blur = mse_loss(img, img_blur)
mse_same = mse_loss(img, img)
mse_img2 = mse_loss(img, img2)
mse_img3 = mse_loss(img, img3)

print(f"{'MSE:':10}  same - {mse_same:.4f}, noise - {mse_noise:.4f}, blur - {mse_blur:.4f}, img2 - {mse_img2:.4f}, img3 - {mse_img3:.4f}")

mse_features_noise = mse_loss(features_img, features_noise)
mse_features_blur = mse_loss(features_img, features_blur)
mse_features_same = mse_loss(features_img, features_img)
mse_features_img2 = mse_loss(features_img, features_img2)
mse_features_img3 = mse_loss(features_img, features_img3)

print(f"{'LPIPS:':10}  same - {mse_features_same:.4f}, noise - {mse_features_noise:.4f}, blur - {mse_features_blur:.4f}, img2 - {mse_features_img2:.4f}, img3 - {mse_features_img3:.4f}")



## Binary File Reading

In [ ]:
from dataset_utils import dataset_generator_from_bag, dataset_generator_from_binary, save_samples_to_disk
import time
import torch
import numpy as np

bag_file_path = r"G:\VM\Shared Folder\bags\COCO\000000000139.bag"
bin_file_path = r"G:\VM\Shared Folder\bags\COCO\000000000139.bin"


bag_gen = dataset_generator_from_bag(bag_file_path, events_topic="/cam0/events", image_topic="/cam0/image_raw")
save_samples_to_disk(bag_gen, "test_bag_batches", False)
bag_gen = dataset_generator_from_bag(bag_file_path, events_topic="/cam0/events", image_topic="/cam0/image_raw")
save_samples_to_disk(bag_gen, "test_bag_batches_compressed", True)
bin_gen = dataset_generator_from_binary(bin_file_path)
save_samples_to_disk(bin_gen, "test_bin_batches", False)
bin_gen = dataset_generator_from_binary(bin_file_path)
save_samples_to_disk(bin_gen, "test_bin_batches_compressed", True)

for i, (bagf, binf) in enumerate(zip(bag_gen, bin_gen)):
    events_equal = np.array_equal(bagf[0], binf[0])
    img_equal = np.array_equal(bagf[1], binf[1])
    print(events_equal, img_equal)

## Colors channeling (scrapped for now)

In [ ]:
import numpy as np
import torch

batch = 2
bins = 2
w = 16
h = 16

x = torch.arange(0, batch*bins*w*h) + 1
x = x.reshape(batch, bins, h, w)

# x = x.reshape(batch, bins, h // 4, w // 4, 2, 2, 2, 2)

r = x[:, :, ::2, ::2]
g = x[:, :, ::2, 1::2]
G = x[:, :, 1::2, ::2]
b = x[:, :, 1::2, 1::2]
x = torch.stack((r,g,G,b), dim=2).squeeze()
print(x.shape, x)

# x = x.transpose(4, 5)
# x = x.transpose(5, 7)

# x = x.reshape(batch, bins, 4, h // 2, w // 2)
# x = x.permute(0, 1, 5, 3, 2, 4)